In [ ]:
import joblib
import torch
import numpy as np
from transformers import BertModel, BertTokenizer

In [ ]:
loaded_model = joblib.load('data/javasbert99.joblib')

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

new_url = "url"

def extract_features(url):
    inputs = tokenizer.encode_plus(url, return_tensors='pt', add_special_tokens=True, max_length=512, truncation=True)

    input_ids = inputs['input_ids']

    if 'attention_mask' in inputs:
        attention_mask = inputs['attention_mask']
    else:
        attention_mask = None

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        hidden_states = outputs[2]

    token_vecs = [torch.mean(hidden_states[layer][0], dim=0) for layer in range(-4, 0)]
    return torch.stack(token_vecs).numpy()

new_features = extract_features(new_url).reshape(1, -1)

if isinstance(loaded_model, torch.nn.Module):
    with torch.no_grad():
        new_features_tensor = torch.tensor(new_features, dtype=torch.float32)
        loaded_model.eval()
        output = loaded_model(new_features_tensor)
        _, prediction = torch.max(output, 1)
        prediction = prediction.item()
else:
    prediction = loaded_model.predict(new_features)

print("Prediction:", prediction)